# Run a Tool from Arachne Python Interface

In this notebook, we explain how to use Arachne Python interface for running a tool.
Like the example for Arachne CLI, we will be working with a tool of TVM to compile ResNet-50 v2 from the Tensorflow Keras Applications. 

## Prepare a Model

First, we prepare an input model by using a Tensorflow Keras API.

In [1]:
import tensorflow as tf

model = tf.keras.applications.resnet_v2.ResNet50V2()
model.summary()
model.save("/tmp/resnet50-v2.h5")

Model: "resnet50v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
_________________________________________________________________________________________

/workspaces/arachne/.venv/lib/python3.6/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


## Initialize `arachne.data.Model`

In Arachne, DNN models are managed by `arachne.data.Model` data objects which keeps the model file path and the tensor specification of the model.
Each tool takes the object as its input and outputs another `Model` as a result.
`arachne.utils.model_utils.get_model_spec` is a helper function to get the tensor specification of the passed model.

In [2]:
from arachne.data import Model
from arachne.utils.model_utils import get_model_spec


model_file_path = "/tmp/resnet50-v2.h5"
input = Model(path=model_file_path, spec=get_model_spec(model_file_path))

print(input)

Model(path='/tmp/resnet50-v2.h5', spec=ModelSpec(inputs=[TensorSpec(name='input_1', shape=[-1, 224, 224, 3], dtype='float32')], outputs=[TensorSpec(name='predictions/Softmax:0', shape=[-1, 1000], dtype='float32')]))


## Deal with the Dynamic Shape

According to the example of the Arachne CLI, we have to deal with the dynamic shape (i.e., `-1` in the shape).
Users can modify the tensor specification directly by updating the value.

In [3]:
input.spec.inputs[0].shape = [1, 224, 224, 3]
input.spec.outputs[0].shape = [1, 1000]
print(input)

Model(path='/tmp/resnet50-v2.h5', spec=ModelSpec(inputs=[TensorSpec(name='input_1', shape=[1, 224, 224, 3], dtype='float32')], outputs=[TensorSpec(name='predictions/Softmax:0', shape=[1, 1000], dtype='float32')]))


## Execute a Tool from Arachne Python Interface

The module for each tool is defined under `arachne.tools`.
In this example, the module for the TVM is implemented at `arachne.tools.tvm`.
There are two main classes.
First, `TVMConfig` is a class for configuraing the behavior of the TVM compile processing.
Next, `TVM` is a wrapper class for executing `tvm.relay.build`.
To run the TVM, users will call a static method (i.e, `TVM.run`) with passing the input model and the config object.

In [4]:
from arachne.tools.tvm import TVMConfig, TVM

tvm_cfg = TVMConfig()
output = TVM.run(input=input, cfg=tvm_cfg)

conv2d NHWC layout is not optimized for x86 with autotvm.
conv2d NHWC layout is not optimized for x86 with autotvm.
conv2d NHWC layout is not optimized for x86 with autotvm.
conv2d NHWC layout is not optimized for x86 with autotvm.
conv2d NHWC layout is not optimized for x86 with autotvm.
conv2d NHWC layout is not optimized for x86 with autotvm.
conv2d NHWC layout is not optimized for x86 with autotvm.
conv2d NHWC layout is not optimized for x86 with autotvm.
conv2d NHWC layout is not optimized for x86 with autotvm.
conv2d NHWC layout is not optimized for x86 with autotvm.
conv2d NHWC layout is not optimized for x86 with autotvm.
conv2d NHWC layout is not optimized for x86 with autotvm.
conv2d NHWC layout is not optimized for x86 with autotvm.
conv2d NHWC layout is not optimized for x86 with autotvm.
conv2d NHWC layout is not optimized for x86 with autotvm.
conv2d NHWC layout is not optimized for x86 with autotvm.
conv2d NHWC layout is not optimized for x86 with autotvm.
conv2d NHWC la

## Save the Result

To save the result as a TAR file, `arachne.utils.model_utils.save_model` should be called.

In [5]:

from arachne.utils.model_utils import save_model

save_model(model=output, output_path="/tmp/output.tar", tvm_cfg=tvm_cfg)

### Pre-defined Configs for TVM Target

To use pre-defined configs for some TVM target, you can use `arachne.tools.tvm.get_predefined_config`.

In [1]:
from arachne.tools.tvm import get_predefined_config

conf = get_predefined_config("dgx-1")
print(conf)

TVMConfig(cpu_target='x86-64', cpu_attr=['+fma', '+avx2'], cpu_name='broadwell', cuda_target_device='nvidia/nvidia-v100', composite_target=['tensorrt', 'cuda'], target=None, target_host=None, desired_layout=None, disabled_pass=None, opt_level=3, export_format='tar', cross_compiler=None, cross_compiler_options=None)
